# CNN을 이용한 이미지 분류
- 학습 내용
  - 손글씨 숫자 이미지 데이터 전처리
    - 색상 차원 추가
    - 정규화 (0-255 -> 0.0-1.0
  - Conv2D(), MaxPooling2D(), Flatten() 사용
  - 손글씨 숫자 분류

In [2]:
from tensorflow.keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
X_train.shape, X_test.shape

((60000, 28, 28), (10000, 28, 28))

In [4]:
# 훈련데이터 2000개, 테스트데이터 500개만 사용
X_train = X_train[:2000]
y_train = y_train[:2000]
X_test = X_test[:500]
y_test = y_test[:500]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((2000, 28, 28), (2000,), (500, 28, 28), (500,))

- 데이터 전처리
  - 색상 차원 추가 (CNN은 색상차원이 필요)
  - 정규화 (0-255 -> 0.0-1.0) : 분산 감소 -> 성능 개선

In [5]:
# 색상차원 추가
# (2000, 28, 28), (500,28,28) -> (2000, 28, 28,1), (500,28,28,1)
# 1을 하나 더 추가하는정도는 할 수 있음. 단순 차원하나 늘리는거라서
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1,28,28,1)

X_train.shape, X_test.shape

((2000, 28, 28, 1), (500, 28, 28, 1))

In [6]:
# 정규화. 최댓값으로 나눔
X_train = X_train / 255.0
X_test = X_test / 255.0

- y를 원핫인코딩. 왜하는지  5-1일 영상을 다시 봐바.. 마지막에서2번째 교시임

In [7]:
# to_categorical() : 원핫인코딩하는 함수. 숫자로된 값만 가능
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

y_train.shape, y_test.shape

((2000, 10), (500, 10))

# CNN을 이용한 신경망 설계
- Conv2D(): 데이터로부터 특성을 추출하는 기능
  - 데이터에 필터를 이용하여 컨볼루션 수행
  - filters: 필터의 수
  - kernel_size: 필터의 크기
  - input_shape: 입력 데이터의 크기(입력층만 설정하면 됨)
  - padding: same으로 하면 크기가 고정
- MaxPooling2D(): 추출된 특성의 크기를 줄이는 기능
  - pool_size: 표본 대사의 크기 (2,2)이면 4배로 크기가 감소
- Flatten(): 특성 데이터를 1차원으로 만드는 기능

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Flatten, Dense

model = Sequential()

# 특성추출기
# C (Conv2D), M(Maxpooling) 층을 쌓는 방법: C-M, C-C-M, C-C-C-M
# 신경망층을 몇 층을 쌓을까: 최종크기?가 4~20 범위가 되기까지 쌓음?
model.add(Conv2D(filters=32, kernel_size=(3,3),
                 input_shape=(28,28,1),
                 padding='valid')) # 제로패딩 안함. 크기 줄어듦. 디폴트값
model.add(Conv2D(filters=64, kernel_size=(3,3),
                 padding='same')) # 크기 줄어듦, 대부분 이거씀
model.add(MaxPooling2D(pool_size=(2,2)))

# 1차원으로 변환
model.add(Flatten())

# 분류기
model.add(Dense(128, activation="relu"))
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_3 (Conv2D)           (None, 26, 26, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 13, 13, 64)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 10816)             0         
                                                                 
 dense_2 (Dense)             (None, 128)               1384576   
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                      

In [10]:
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

- 베스트모델 저장, 학습중단 기능

- 학습

In [11]:
h1 = model.fit(X_train, y_train, batch_size=100,
               epochs=30, validation_data=[X_test, y_test])

Epoch 1/30
20/20 [==============================] - 7s 283ms/step - loss: 0.8944 - accuracy: 0.7325 - val_loss: 0.4752 - val_accuracy: 0.8480
Epoch 2/30
20/20 [==============================] - 5s 237ms/step - loss: 0.3014 - accuracy: 0.9060 - val_loss: 0.3448 - val_accuracy: 0.8860
Epoch 3/30
20/20 [==============================] - 6s 306ms/step - loss: 0.1676 - accuracy: 0.9505 - val_loss: 0.2871 - val_accuracy: 0.9060
Epoch 4/30
20/20 [==============================] - 5s 253ms/step - loss: 0.1029 - accuracy: 0.9705 - val_loss: 0.2568 - val_accuracy: 0.9180
Epoch 5/30
20/20 [==============================] - 6s 327ms/step - loss: 0.0580 - accuracy: 0.9850 - val_loss: 0.2842 - val_accuracy: 0.9240
Epoch 6/30
20/20 [==============================] - 5s 239ms/step - loss: 0.0256 - accuracy: 0.9960 - val_loss: 0.2811 - val_accuracy: 0.9240
Epoch 7/30
20/20 [==============================] - 5s 247ms/step - loss: 0.0135 - accuracy: 0.9980 - val_loss: 0.2932 - val_accuracy: 0.9180
Epoch 